In [17]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [53]:
driver = webdriver.Chrome(r'C:\Users\Asad\seowork\chromedriver.exe')

In [55]:
# Initialising empty lists    
size = [] 
desc = []
sale = []
location = []
locations = ['london','reading','maidenhead']
for x in locations:
        p= 1
        while True:
            url = 'https://www.novaloca.com/property-search-results/default.aspx?page={}&propertytypes=&saletypes=1&status=1&sizefrom=1&sizeto=500000%2b&sizetype=2&location={}&distance=30&addresskeyword=&excludeServicedOffices=False'.format(p,x)
            driver.get(url)
            time.sleep(3)
            location.append(x)
            p += 1
            
            try:
                # Test to see if we're on the last page
                test = driver.find_element_by_class_name('extra-details')

#                 print(f'this is {p} of {x}')

                # Execution Code For Every Page
                ids = driver.find_elements_by_xpath("//*[contains(@id,'property-')]") # Using Regex to extract all ids
                property_ids = []
                for i in ids:
                    property_ids.append(i.get_attribute('id'))
                    for x in property_ids:
                    #     To extract sizes from each property-card
                        size_test = driver.find_elements_by_xpath('//*[@id="'+x+'"]/div/div[1]/div[3]/div/div[1]/div/span/strong')[0]    
                        size.append(size_test.text)
                        desc_test = driver.find_elements_by_xpath('//*[@id="'+x+'"]/div/div[2]/h4')[0]
                        desc.append(desc_test.text)
                        sale_test = driver.find_elements_by_xpath('//*[@id="'+x+'"]/div/div[2]/p/span[2]')[0]
                        sale.append(sale_test.text)

            except:
                break





In [56]:
df_data = pd.DataFrame({'prop_size':size,'prop_desc':desc,'prop_price':sale})


In [57]:
df_data

,prop_size,prop_desc,prop_price
0,"1,695 to 4,645 Sq Ft","Office / Residential Opportunity, 40 Great Smi...","- £3,400,000.00"
1,"1,695 to 4,645 Sq Ft","Office / Residential Opportunity, 40 Great Smi...","- £3,400,000.00"
2,"5,690 Sq Ft","B1 Unit or May Suit A1, A2, D1, D2 (STP), Pala...","- £2,250,000.00"
3,"1,695 to 4,645 Sq Ft","Office / Residential Opportunity, 40 Great Smi...","- £3,400,000.00"
4,"5,690 Sq Ft","B1 Unit or May Suit A1, A2, D1, D2 (STP), Pala...","- £2,250,000.00"
...,...,...,...
970,"15,715 to 32,140 Sq Ft","For Sale - 3 REFURBISHED INDUSTRIAL 15,715 - 3...",
971,"15,715 to 32,140 Sq Ft","Wye Business Park, Thomas Road, Wooburn Green,...",
972,"45,176 Sq Ft",Self-Contained Industrial/Warehouse with Large...,
973,"4,090 Sq Ft","Commercial Premises With A1, A3 or D2 Use, Win...",
